In [4]:
import numpy as np
import pandas as pd
from helpfiles.temp_save_load import save_files, load_files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import helpfiles.ml_pipeline_config as configurations

#Load data
df = pd.read_csv("C:/Users/PC Paul/Downloads/DataEngOwnProject/data/creditcard.csv")#.head(10000) 
while len(df.index) < 1000000: # Todo: uncomment
    df = pd.concat([df,df])
df.reset_index(drop = True, inplace = True)

#scale_data():
rob_scaler = RobustScaler()
df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(['Time','Amount'], axis=1, inplace=True)


#split_data_train_val():
split_ratio = configurations.params['test_split_ratio']
# Splitting
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# Dataframe is concatenated again as the full dataset is needed and split again during cross validation. _val dataframes are needed for the final evaluation. 
df = pd.concat([X_train, y_train],axis = 1)

#undersample_data():
df2 = df.sample(frac=1) # this shuffles the initial df
# Filter fraud cases and concat equal amount of non-fraud-cases
fraud_df = df2.loc[df['Class'] == 1]
non_fraud_df = df2.loc[df['Class'] == 0][:len(fraud_df)]
normal_distributed_df = pd.concat([fraud_df, non_fraud_df])
# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

In [5]:
new_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class
355396,-2.042608,1.573578,-2.372652,-0.572676,-2.097353,-0.174142,-3.039520,-1.634233,-0.594809,-5.459602,...,0.501222,-0.696892,-0.600514,0.127547,-0.786072,0.606097,0.171697,3.351544,-0.361313,1
681343,-0.508574,0.326057,2.258090,0.085603,-1.266965,-0.218649,0.147233,0.069141,0.659956,-0.635383,...,-0.091377,0.057221,0.784309,-0.689903,0.940863,-0.013294,0.124774,0.661590,-0.145078,0
735224,1.956552,-0.058671,-1.036132,1.291854,0.176632,-0.164698,-0.000702,-0.058953,0.466270,0.320406,...,0.706556,-0.039530,-0.374002,0.331067,-0.456771,0.022007,-0.063080,-0.167808,0.385867,0
758678,1.546322,-0.754504,-2.370798,0.106949,1.042846,1.300035,-0.062101,0.397464,0.841893,-0.937829,...,-0.335966,0.047470,-1.077996,-0.283541,-0.025376,-0.003459,-0.007373,2.444600,0.511713,0
407288,-0.500881,0.268217,1.981096,-1.316633,-0.762119,-0.462187,-0.093959,0.161073,1.460445,-1.391160,...,1.112432,-0.156227,0.424759,-0.399876,-0.639136,0.116901,0.164423,-0.292022,-0.095348,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844089,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,-1.745854,-2.516628,...,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,0.250105,0.955545,1
1005108,-10.300820,6.483095,-15.076363,6.554191,-8.880252,-4.471672,-14.900689,3.840170,-4.358441,-14.533162,...,1.041642,-0.682790,0.573544,-1.602389,-0.393521,-0.468893,0.105920,-0.293419,0.108141,1
724332,-5.603690,5.222193,-7.516830,8.117724,-2.756858,-1.574565,-6.330343,2.998419,-4.508167,-7.334377,...,0.428408,-0.101184,-0.520199,-0.176938,0.461450,-0.106625,-0.479662,-0.307391,0.211196,1
327356,-5.314173,4.145944,-8.532522,8.344392,-5.718008,-3.043536,-10.989185,3.404129,-6.167234,-11.435624,...,0.862996,-0.614453,0.523648,-0.712593,0.324638,2.245091,0.497321,0.925388,-0.511572,1


In [7]:
#from sklearn.decomposition import PCA, TruncatedSVD

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
#from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

import helpfiles.ml_pipeline_config as configurations

#experiment():
X = new_df.drop('Class', axis=1)
y = new_df['Class']

#classifiers = {
#"LogisticRegression": LogisticRegression(),
#"Support Vector Classifier": SVC()
#}

#for key, classifier in classifiers.items():
#classifier.fit(X, y)
#training_score = cross_val_score(classifier, X, y, cv=5, scoring = 'f1')
#print("Classifiers: ", classifier.__class__.__name__, "Has a training F1 of", round(training_score.mean(), 2) * 100)


# Logistic Regression 
log_reg_maxiter = configurations.params["logreg_maxiter"]
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} #todo: put this into pipeline config
# Grid search
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
#grid_log_reg = GridSearchCV(LogisticRegression(max_iter = log_reg_maxiter), log_reg_params) # todo: see whether you can use this line
grid_log_reg.fit(X, y)
# Logistic Regression estimator:
log_reg = grid_log_reg.best_estimator_

# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']} #todo: put this into pipeline config
# Grid search
grid_svc = GridSearchCV(SVC(), svc_params)
grid_svc.fit(X, y)
# SVC best estimator
svc = grid_svc.best_estimator_

# Evaluation
# Logistic Regression 
log_reg_score = cross_val_score(log_reg, X, y, cv=5, scoring = 'f1')
print('Logistic Regression Cross Validation F1 score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters Logistic Regression: " + str(grid_log_reg.best_params_))
# SVC
svc_score = cross_val_score(svc, X, y, cv=5, scoring = 'f1')
print('Support Vector Classifier Cross Validation F1 score', round(svc_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters SVM: " + str(grid_svc.best_params_))

#X_val = load_files(['X_val'])[0]
#y_val = load_files(['y_val'])[0]

#Predict Y
y_pred_log_reg = log_reg.predict(X_val)
y_pred_SVM = svc.predict(X_val)
f1_log_reg = f1_score(y_val, y_pred_log_reg)
f1_SVM = f1_score(y_val, y_pred_SVM)

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

log_reg_results = pd.DataFrame([[
    now,
    'Logistic Regression',
    grid_log_reg.best_estimator_,
    grid_log_reg.best_params_,
    f1_log_reg
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])
log_reg_results

svc_results = pd.DataFrame([[
    now,
    'Support Vector Classifier',
    grid_svc.best_estimator_,
    grid_svc.best_params_,
    f1_SVM
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])

results = pd.concat([log_reg_results,svc_results])
    


Logistic Regression Cross Validation F1 score:  94.48%
Best Parameters Logistic Regression: {'C': 100, 'penalty': 'l2'}
Support Vector Classifier Cross Validation F1 score 94.63%
Best Parameters SVM: {'C': 1, 'kernel': 'linear'}
